In [ ]:
"""
# PCA
# TSNE
# RBM


机器学习 本质上就是对于数据进行提取 吸收的过程
根据待处理数据的性状 
具备标签特质的
有标签
监督学习 分类与回归
无标签
非监督学习 聚类 降维
不具备标签特质的
强化学习

根据对应的算法可以分为几类:
判别式 不关注概率分布本身,只关注是否成功区分数据
生成式,关注概率分布,构建虚拟的数据分布, 可以从新分布中采样,和判别
真实概率 -> 判别结果
真实概率 -> 模拟概率 -> 判别结果

什么是好数据:
1 从表面, 数据不重不漏 没有缺损, 规则美
2 从统计, 数据的特征可以覆盖数据本身和问题所需,充分必要
由此引入的技术手段
数据清洗
数据的预处理   降维 维度爆炸

整体流程
数据特征定义 -> 数据采集 -> 数据清洗 -> 数据归纳 -> 数据存储 -> AI
所有事物,或抽象或具体,都可以构建为一个特定的概率分布P(X)
AI 流程
生成式: 构建概率 -> 因式分解优化 -> 最小概率构建神经网络 -> 优化网络结构,调参,训练 -> 优化部署
判别式: 构建神经网络 ->  调参,训练 -> 优化部署

图神经网络GNN
比如交通预测和多模态预测,还深入了解动态时空特征和潜在偶然性。
最近的一些研究表明,将序列周期性或频率处理纳入时间序列 Transformer 架构将带来卓越的性能改进。

按派别 
频率派认为θ 是一个未知的常量
贝叶斯派认为是一个随机变量

判别式模型

生成式模型
概率图模型

指数族分布


![](imgs/1.png)
![](imgs/2.png)
η(yita) 规范化参数 
A(η) log partition function (log 配分函数) 
3
![](imgs/3.png)
Z 归一化因子 配分函数

指数族分布特点 
充分统计量 (对样本的充分加工) : 在线学习 online learning
共轭
最大熵
为什么指数族分布具有充分统计量的特点
4
![](imgs/4.png)
最大熵, 无信息先验
指数族分布
广义线性模型 
概率图模型
变分推断
"""
import pandas as pd

class Autogluon():
    def __init__(self,id,label,model_save_path):
        """
        table类的数据,可以用于快速的基准测试
        # !pip install "mxnet<2.0.0"
        # !pip install autogluon

        :param id:
        :param label:
        :param model_save_path:
        """
        self.id = id
        self.label = label
        self.model_save_path = model_save_path
        if id is None:
            raise 'id不可以为None 请设置id列'
        if label is None:
            raise 'label不可以为None,请设置label列'

    def fit(self,traindata):
        from autogluon import TabularDataset, TabularPredictor
        train_data = TabularDataset(traindata)
        self.predictor = TabularPredictor(label=self.label, path=self.model_save_path).fit(train_data.drop(columns=[self.id]))

    def predict(self,testdata):
        from autogluon import TabularDataset, TabularPredictor
        test_data = TabularDataset(testdata)
        if self.label in test_data.columns:
            preds = self.predictor.predict(test_data.drop(columns=[self.id,self.label]))
        else:
            preds = self.predictor.predict(test_data.drop(columns=[self.id]))
        return pd.DataFrame({self.id: test_data[self.id], self.label: preds})

    def accuary(self,testlabel,submission):
        pass


__all__ = ['Autogluon']







## 遗传算法

import pandas as pd
import pygad
import numpy as np

imbdata = pd.read_csv('datas.csv')


def MT(A_f, p, H, D_max, D_min, D_c, FD, sa, a, yip, theta, miu, t):
    part1 = (A_f * p) / 2 * D_max ** (2 - FD)
    part2 = sa / (1 - (D_min / D_max) ** (2 - FD))
    part3 = np.sqrt(yip * np.cos(theta) / miu) * (2 - FD) / (2.5 - FD)
    part4 = (D_c ** (2.5 - FD) - D_min ** (2.5 - FD)) * np.sqrt(t)
    part5 = A_f * p * H / D_max ** (2 - FD) * a / (1 - (D_min / D_max) ** (2 - FD)) * (
            D_max ** (2 - FD) - D_c ** (2 - FD))
    # return part1 * part2 * part3 * part4 + part5
    return part5


def MT2(alpha, t, FD):
    D_c = 0.154538803 * alpha ** 2 / t
    sa = 0.016292895 * alpha
    D_min = 0.016863889 * alpha ** 2
    D_max = 36194.5

    D_c = np.clip(D_c, D_min, D_max)
    #     if D_c>D_max:
    #     D_c = D_max
    # elif D_c<D_min:
    #     D_c = D_min

    return MT(A_f=0.6666, p=0.7342,
              H=1.03,
              D_max=D_max,
              a=0.016292895,
              yip=0.87,
              theta=0,
              miu=23.89,
              t=t,
              FD=FD,
              D_min=D_min,
              D_c=D_c,
              sa=sa,
              )


def OF(alpha=1, FD=3, imbdata=imbdata):
    end_list = []
    for i in range(imbdata.shape[0]):
        t = imbdata.iloc[i, :]['t']
        mexp = imbdata.iloc[i, :]['Mexp']
        end_list.append(np.abs((MT2(alpha=alpha, t=t, FD=FD) - mexp) / mexp))
    return np.mean(end_list)

def fitness_func(solution, solution_idx):
    x = solution[0]
    y = solution[1]
    return OF(alpha=x,FD=y)

def min_fitness_func(solution, solution_idx):
    return -1 * fitness_func(solution, solution_idx)

num_generations = 500  # 遗传算法的迭代次数
num_parents_mating = 10 # 每一代选择的父代数量
sol_per_pop = 200  # 每一代的种群数量
num_genes = 2  # 每个染色体的基因数量 （参数个数）
gene_space = [{"low": 2, "high": 4.5},{"low": 1, "high": 2}] # 基因空间的范围 （参数范围）
ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       gene_type=float,
                       init_range_low=1,
                       init_range_high=4,
                       gene_space=gene_space,
                       fitness_func=min_fitness_func
                      )


ga_instance.run()

solution,solution_fitniss,other = ga_instance.best_solution()

solution,solution_fitniss,other

ga_instance.plot_result()

alpha=4.49
FD=1.99
OF(alpha,FD)
f"alpha:{alpha} FD:{FD} OF:{OF(alpha,FD)}"


# TODO 遗传算法
1.
已知参数
Af = 0.6666
ρ = 0.7342
θ = 0
σ = 0.87
μ = 23.89
Dmax = 36194.5
Φa = 0.016292895
2.
未知参数
Dmin =
Φsa =
Dc =

3.
输入值
t与Mexp
（数值见表格）

4.
输出值

N为t数值的个数
OF值最小时，对应的a与FD即为所求

import numpy as np


def MT(A_f, p, H, D_max, D_min, D_c, FD, sa, a, yip, theta, miu, t):
    part1 = (A_f * p) / (2 * D_max) ** (2 - FD)
    part2 = sa / (1 - (D_min / D_max) ** (2 - FD))
    part3 = np.sqrt(yip * np.cos(theta) / miu) * (2 - FD) / (2.5 - FD)
    part4 = (D_c ** (2.5 - FD) - D_min ** (2.5 - FD)) * np.sqrt(t)
    part5 = A_f * p * H / D_max ** (2 - FD) * a / (1 - (D_min - D_max) ** (2 - FD)) * (
            D_max ** (2 - FD) - D_c ** (2 - FD))
    return part1 * part2 * part3 * part4 + part5


def MT2(alpha, t, FD):
    D_c = 0.154538803 * alpha ** 2 / t
    sa = 0.016292895 * alpha
    D_min = 0.016863889 * alpha ** 2
    D_max = 36194.5

    D_c = np.clip(D_c, D_min, D_max)
    #     if D_c>D_max:
    #     D_c = D_max
    # elif D_c<D_min:
    #     D_c = D_min

    return MT(A_f=0.6666, p=0.7342,
              H=1.03,
              D_max=D_max,
              a=0.016292895,
              yip=0.87,
              theta=0,
              miu=23.89,
              t=t,
              FD=FD,
              D_min=D_min,
              D_c=D_c,
              sa=sa,
              )


def OF(alpha=1, FD=3, imbdata=imbdata):
    end_list = []
    for i in range(imbdata.shape[0]):
        t = imbdata.iloc[i, :]['t']
        mexp = imbdata.iloc[i, :]['Mexp']
        end_list.append(np.abs((MT2(alpha=alpha, t=t, FD=FD) - mexp) / mexp))
    return np.mean(end_list)


# -*-coding:utf-8 -*-
# 目标求解2*sin(x)+cos(x)最大值
import random
import math
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm


class GA(object):
    # 初始化种群 生成chromosome_length大小的population_size个个体的种群

    def __init__(self, population_size, chromosome_length, max_value, min_value, pc, pm):

        self.population_size = population_size
        self.choromosome_length = chromosome_length
        # self.population=[[]]
        self.max_value = max_value
        self.min_value = min_value

        self.pc = pc
        self.pm = pm
        # self.fitness_value=[]

    def species_origin(self):
        """
        生成一个随机的二进制种群。
        """

        population = np.random.randint(2, size=(self.population_size, sum(self.choromosome_length)))
        return population

    # 从二进制到十进制
    # 编码  input:种群,染色体长度 编码过程就是将多元函数转化成一元函数的过程
    def trans2(self, population):
        k = np.cumsum(self.choromosome_length)
        params = []
        for i in range(len(k)):
            if i == 0:

                param = self.translation(population[:, :k[i]])
            else:
                param = self.translation(population[:, k[i - 1]:k[i]])
            params.append(param)
        return params

    def translation(self, population):
        """
        将种群中的二进制染色体转换成对应的十进制数字。
        """
        decimal_population = []
        for chromosome in population:
            decimal_num = 0
            for i in range(len(chromosome)):
                decimal_num += chromosome[i] << i
            decimal_population.append(decimal_num)
        return decimal_population

    # 返回种群中所有个体编码完成后的十进制数

    # from protein to function,according to its functoin value
    # a protein realize its function according its structure
    # 目标函数相当于环境 对染色体进行筛选，这里是2*sin(x)+math.cos(x)

    def function(self, population, func):  # over
        temporary_x, temporary_y = self.trans2(population)
        x_list = np.array(temporary_x) / np.power(2, self.choromosome_length[0]) * (
                    self.max_value - self.min_value) + self.min_value
        y_list = np.array(temporary_y) / np.power(2, self.choromosome_length[1]) * (
                    self.max_value - self.min_value) + self.min_value
        return [func(x_list[i], y_list[i]) for i in range(len(x_list))]

    # 定义适应度
    def fitness(self, function1, threshold=10, upper_bound=True):
        """
        计算适应度，这里假设适应度值为 function1 的值大于/小于 threshold 的值，其他值为上下边界值。

        Args:
        function1: list，要计算适应度的列表
        threshold: float，阈值，大于或小于该值的函数值会被判定为合格或不合格
        upper_bound: bool，是否计算上边界，默认为 True，即计算大于阈值的上边界值

        Returns:
        fitness_value: list，每个函数值对应的适应度值
        """
        fitness_value = []
        for value in function1:
            if upper_bound:
                fitness = threshold if value > threshold else value
            else:
                fitness = threshold if value < threshold else value
            fitness_value.append(fitness)
        return fitness_value

    def selection(self, pop, fitness_value):
        # 计算适应度值
        total_fit = np.sum(fitness_value)
        newfitness_value = fitness_value / total_fit

        # 计算累积概率
        accumulation_value = np.cumsum(newfitness_value)

        # 生成随机数
        ms = np.random.random(len(pop))

        # 选择新个体
        newpop = np.empty_like(pop)
        for i in range(len(pop)):
            j = np.searchsorted(accumulation_value, ms[i])
            newpop[i] = pop[j]

        return newpop

    # 交叉操作
    def crossover(self, population):
        pop_len = len(population)

        for i in range(0, pop_len - 1, 2):
            if np.random.random() < self.pc:
                cpoint = np.random.randint(0, len(population[0]))

                # 单点交叉
                temporary1 = np.concatenate((population[i][0:cpoint], population[i + 1][cpoint:]))
                temporary2 = np.concatenate((population[i + 1][0:cpoint], population[i][cpoint:]))

                population[i] = temporary1
                population[i + 1] = temporary2

        return population

    def mutation(self, population):
        px, py = population.shape

        for i in range(px):
            if np.random.random() < self.pm:
                mpoint = np.random.randint(0, py)
                # 单点随机变异
                population[i, mpoint] = 1 - population[i, mpoint]

        return population

    # def b2d(self, best_individual):
    #     best_individual = np.array(best_individual)
    #     total = np.sum(best_individual * (2 ** np.arange(len(best_individual))))
    #     # 使用广播功能进行向量化操作，避免了不必要的循环和计算
    #     total = total * self.max_value / (2 ** len(best_individual) - 1)
    #     return total.tolist()
    def b2d(self, best_individual):
        # TODO
        params_x, params_y = self.trans2(np.array([best_individual]))
        x_ = np.array(params_x) / np.power(2, self.choromosome_length[0]) * (
                    self.max_value - self.min_value) + self.min_value
        y_ = np.array(params_y) / np.power(2, self.choromosome_length[1]) * (
                    self.max_value - self.min_value) + self.min_value
        return x_, y_

    def best(self, population, fitness_value):
        best_idx = np.argmin(fitness_value)
        best_individual = population[best_idx]
        best_fitness = fitness_value[best_idx]
        return [best_individual, best_fitness]

    def plot(self, results, rang):
        X = []
        Y = []

        for i in range(rang):
            X.append(i)
            Y.append(results[i][0])

        plt.plot(X, Y)
        plt.show()

    def main(self):

        results = [[]]
        fitness_value = []
        fitmean = []

        population = pop = self.species_origin()
        lunshu = 500
        for i in tqdm(range(lunshu)):
            function_value = self.function(population, func=OF)
            fitness_value = self.fitness(function_value)

            best_individual, best_fitness = self.best(population, fitness_value)
            results.append([best_fitness, self.b2d(best_individual)])
            # 将最好的个体和最好的适应度保存，并将最好的个体转成十进制,适应度函数
            population = self.selection(population, fitness_value)
            population = self.crossover(population)
            population = self.mutation(population)
        results = results[1:]
        results.sort()
        self.plot(results, rang=lunshu)
        return results

import pandas as pd
imbdata = pd.read_csv('datas.csv')

population_size=400
max_value=5
min_value=-5
chromosome_length=[5,5]
pc=0.6
pm=0.01
ga=GA(population_size,chromosome_length,max_value,min_value,pc,pm)
result = ga.main()


fitness = np.array(result)[:,0]
index = fitness.tolist().index(fitness.max())
best_params,best_fit = np.array(result)[index,1],np.array(result)[index,0]

f'最优参数:{best_params},'

f'最优结果:{OF(best_params[0][0],best_params[1][0])}'